# Library

In [12]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torch.autograd import Variable
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

from ignite.metrics import mIoU
from torchmetrics.classification import ConfusionMatrix

import numpy as np
import os
# import cv2
from PIL import Image
from tqdm import tqdm_notebook as tqdm
import random
from matplotlib import pyplot as plt
import pandas as pd
from collections import namedtuple
from sklearn.metrics import accuracy_score
# import rasterio
from copy import deepcopy
from sklearn.model_selection import train_test_split

In [13]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

# Dataset

In [14]:
RANDOM_STATE = 42

# Train

### Data load

In [15]:
from sklearn.model_selection import train_test_split
from dataset import CustomDataset
from torch.utils.data import Dataset, DataLoader
import multiprocessing 
#multiprocessing.set_start_method('spawn')
BASE = '.'
train_meta = pd.read_csv(f'{BASE}/train_meta.csv')#.sample(n=30000,random_state=RANDOM_STATE)
test_meta = pd.read_csv(f'{BASE}/test_meta.csv')

# 데이터 위치
IMAGES_PATH = f'{BASE}/train_img/'
MASKS_PATH = f'{BASE}/train_mask/'

# 가중치 저장 위치
SAVE_PATH = f'{BASE}/train_output/'
MODEL_SAVE = f'{SAVE_PATH}/best_UNet_Base_model.pth'

# train : val = 8 : 2 나누기
x_tr, x_val = train_test_split(train_meta, test_size=0.2, random_state=RANDOM_STATE)

# train : val
images_train = [os.path.join(IMAGES_PATH, image) for image in x_tr['train_img'] ]
masks_train = [os.path.join(MASKS_PATH, mask) for mask in x_tr['train_mask'] ]

images_valid = [os.path.join(IMAGES_PATH, image) for image in x_val['train_img'] ]
masks_valid = [os.path.join(MASKS_PATH, mask) for mask in x_val['train_mask'] ]

WORKERS = 8
BATCH_SIZE = 32
train_dataset = CustomDataset(images_train, masks_train, mode='train')
valid_dataset = CustomDataset(images_valid, masks_valid, mode='valid')

train_dataloader = DataLoader(
   dataset=train_dataset,
   batch_size=BATCH_SIZE,
   shuffle=True,
   num_workers=WORKERS,
   pin_memory=True
)

val_dataloader = DataLoader(
   dataset=valid_dataset,
   batch_size=BATCH_SIZE,
   shuffle=False,
   num_workers=WORKERS,
   pin_memory=True
)

In [ ]:
# train_transforms = transforms.Compose([
#             transforms.ToTensor(),
#         ])
# val_transforms = transforms.Compose([
#             transforms.ToTensor(),
#         ])

In [16]:
#WORKERS = 0 #
EPOCHS = 100 # 훈련 epoch 지정
BATCH_SIZE = 32 # batch size 지정
IMAGE_SIZE = (256, 256) # 이미지 크기 지정

# MAX_NORM = 3
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(DEVICE)
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

seed_everything(RANDOM_STATE) # SEED 고정

cuda


### Model train

#### model selection

In [17]:
class DiceLoss(nn.Module):
    def __init__(self):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        inputs = F.sigmoid(inputs) # sigmoid를 통과한 출력이면 주석처리
        
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth) / (inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice 

In [18]:
# torch init cache
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [21]:
def calculate_iou(y_true, y_pred):
    """
    한 클래스에 대한 IoU를 계산합니다.
    """
    intersection = np.logical_and(y_true, y_pred)
    union = np.logical_or(y_true, y_pred)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

def calculate_miou(y_true, y_pred):
    """
    산불이 있는 경우와 없는 경우, 두 클래스에 대한 평균 IoU(mIoU)를 계산합니다.
    """
    y_pred = F.sigmoid(y_pred)
    y_pred = np.where(y_pred > 0.2, 1, 0) # 임계값 처리
    iou_fire = calculate_iou(y_true == 1, y_pred == 1)
    
    # 두 클래스에 대한 IoU 평균 계산
    return iou_fire
def pixel_accuracy (y_true, y_pred):
    low = np.quantile(y_pred, 0.99)
    y_pred = np.where(y_pred >  low, 1, 0) # 임계값 처리
    sum_n = np.sum(np.logical_and(y_pred, y_true))
    sum_t = np.sum(y_true)

    if (sum_t == 0):
        pixel_accuracy = 0
    else:
        pixel_accuracy = sum_n / sum_t
    return pixel_accuracy

In [ ]:
import segmentation_models_pytorch as smp
model = smp.Unet('timm-efficientnet-b3', encoder_weights='imagenet',  classes=1, activation=None,in_channels=1)#, encoder_depth=5, decoder_channels=[256, 128, 64, 32, 16])
#model.load_state_dict(torch.load(MODEL_SAVE))

In [ ]:
# from transformers import MaskFormerConfig, MaskFormerForInstanceSegmentation
# from datasets import load_dataset
# # MaskFormer 모델 구성 수정
# config = MaskFormerConfig.from_pretrained("facebook/maskformer-swin-base-ade")
# config.num_labels  = 1 
# config.mask_feature_size = 1
# model = MaskFormerForInstanceSegmentation(config)

In [ ]:
#model = smp.DeepLabV3Plus('resnet34', encoder_weights='imagenet',classes=1, activation='sigmoid',in_channels = 1)

In [ ]:
# model = smp.Unet(encoder_name='efficientnet-b0',encoder_weights ='imagenet',in_channels=10, classes=1,activation='sigmoid',)
# weight_decay = 1e-4
optimizer = optim.AdamW(model.parameters(), lr=0.001)#(params=model.parameters(), lr=0.0001)
loss_fn = nn.BCEWithLogitsLoss().to(DEVICE)
# loss_fn = DiceLoss().to(DEVICE)

In [ ]:
# loss_fn = torchvision.ops.sigmoid_focal_loss

In [ ]:
lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0.000001)

In [ ]:
best_val_loss = 0
best_model = None
early_stop = 0

model.to(DEVICE)
print('Train START')
for epoch in range(1, EPOCHS+1):
    model.train()
    print('Epoch {}/{}'.format(epoch, EPOCHS))
    print('-'*20)
    train_loss, train_accs, train_ious = 0, [], []

    #### train ####
    for imgs, masks in tqdm(train_dataloader):
        imgs = imgs.to(DEVICE)
        masks = masks.to(DEVICE)
        
        optimizer.zero_grad()  # 초기화
        output = model(imgs)  # 예측
        loss = loss_fn(output, masks)  # 순전파

        loss.backward()  # 역전파
        optimizer.step()  # 학습
        
        iou = calculate_miou(masks.cpu().detach().numpy(), output.cpu())
        output = output.cpu().detach().numpy()
        masks = masks.cpu().detach().numpy()
        accuracy = pixel_accuracy(masks, output)  # mIou
        train_accs.append(accuracy)
        train_ious.append(iou)
        train_loss += loss.item()

    #### valid ####
    model.eval()
    val_loss, val_accs, val_ious = 0, [], []
    with torch.no_grad():
        for imgs, masks in tqdm(val_dataloader):
            imgs = imgs.to(DEVICE)
            masks = masks.to(DEVICE)

            output = model(imgs)  # 예측

            loss = loss_fn(output, masks)  # 순전파
            iou = calculate_miou(masks.cpu().detach().numpy(), output.cpu())
            output = output.cpu().detach().numpy()
            masks = masks.cpu().detach().numpy()
            accuracy = pixel_accuracy(masks, output)  # mIou
            val_accs.append(accuracy)
            val_ious.append(iou)
            val_loss += loss.item()

    train_acc = np.mean(train_accs)
    train_iou = np.mean(train_ious)
    val_acc = np.mean(val_accs)
    val_iou = np.mean(val_ious)
    val_loss /= len(val_dataloader)  # 평균 계산

    print(f"EPOCH: {epoch}, TRAIN LOSS: {train_loss:.6f}, TRAIN mIou: {train_iou:.6f}, TRAIN ACC: {train_acc:.6f}, VAL LOSS: {val_loss:.6f}, VAL mIou: {val_iou:.6f}, VAL ACC: {val_acc:.6f}")

    if lr_scheduler is not None:
        lr_scheduler.step(val_loss)

    if best_val_loss < val_iou:
        print("Model Save")
        best_val_loss = val_iou
        torch.save(model.state_dict(), MODEL_SAVE)
        early_stop = 0
    else:
        early_stop += 1

    # early stop
    if early_stop > 10:
        print("Early Stop")
        break

In [ ]:
NUM = 4

import numpy as np
import matplotlib.pyplot as plt
with torch.no_grad():
    masksx = masks[NUM].cpu().detach().numpy()
    masksx = np.reshape(masksx,(256,256,1))
    # 이미지 표시
    plt.imshow(masksx, cmap='gray')  # 흑백 이미지로 표시
    plt.axis('off')  # 축 제거
    plt.show()

    image = imgs[NUM].cpu().detach().numpy()
    image = np.reshape(image,(256,256,1))
    # 이미지 표시
    plt.imshow(image)  # 흑백 이미지로 표시
    plt.axis('off')  # 축 제거
    plt.show()
    output = model(imgs)
    masksx2 = output[NUM].cpu()#.detach().numpy()

    masksx2=F.sigmoid(masksx2)
    masksx2 = np.where(masksx2 > 0.2, 1, 0) # 임계값 처리
    masksx2 = np.reshape(masksx2,(256,256,1))
    # 이미지 표시
    plt.imshow(masksx2, cmap='gray')  # 흑백 이미지로 표시
    plt.axis('off')  # 축 제거
    plt.show()
    print(np.unique(masksx))
    a = np.where(masksx == 1)
    b = np.where(masksx2 == 1)
    print(f'a:{a}')
    print(f'b:{b}')

In [22]:
import segmentation_models_pytorch as smp
model1 = smp.Unet('timm-efficientnet-b3', encoder_weights='imagenet',  classes=1, activation=None,in_channels=1)
model2 = smp.Unet('timm-efficientnet-b3', encoder_weights='imagenet',  classes=1, activation=None,in_channels=1)
model3 = smp.Unet('timm-efficientnet-b3', encoder_weights='imagenet',  classes=1, activation=None,in_channels=1)
model4 = smp.Unet('timm-efficientnet-b3', encoder_weights='imagenet',  classes=1, activation=None,in_channels=1)
model5 = smp.Unet('timm-efficientnet-b3', encoder_weights='imagenet',  classes=1, activation=None,in_channels=1)
model6 = smp.Unet('timm-efficientnet-b3', encoder_weights='imagenet',  classes=1, activation=None,in_channels=1)
model7 = smp.Unet('timm-efficientnet-b3', encoder_weights='imagenet',  classes=1, activation=None,in_channels=1)
loss_fn = nn.BCEWithLogitsLoss().to(DEVICE)

In [23]:
import torch
import numpy as np
from tqdm import tqdm

# 모델 리스트 정의
models = [model1.to(DEVICE), model2.to(DEVICE), model3.to(DEVICE), model4.to(DEVICE), model5.to(DEVICE), model6.to(DEVICE), model7.to(DEVICE)]  # 실제 모델 객체로 대체해야 합니다.

# 모델 저장 경로 리스트 정의
model_save_paths = [
    f'{SAVE_PATH}/best_model1.pth',
    f'{SAVE_PATH}/best_model2.pth',
    f'{SAVE_PATH}/best_model3.pth',
    f'{SAVE_PATH}/best_model4.pth',
    f'{SAVE_PATH}/best_model5.pth',
    f'{SAVE_PATH}/best_model6.pth',
    f'{SAVE_PATH}/best_model7.pth'
]
optimizers = [torch.optim.AdamW(model.parameters(), lr=0.001) for model in models]
lr_schedulers = [optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0.000001) for optimizer in optimizers]
best_val_losses = [0] * len(models)
early_stops = [0] * len(models)
# 각 모델에 대한 학습 진행
print('Train START')
for epoch in range(1, EPOCHS+1):
    for model in models:
        model.train()
    print('Epoch {}/{}'.format(epoch, EPOCHS))
    print('-'*20)
    train_losses, train_accs, train_ious = [0] * len(models), [[] for _ in range(len(models))], [[] for _ in range(len(models))]
    #### train ####
    for images, masks_tr in tqdm(train_dataloader):
        for idx, (model, optimizer) in enumerate(zip(models, optimizers)):
            imgs =images[:,:,:,(idx)]
            imgs =np.reshape(imgs, (BATCH_SIZE,1,256,256)).to(DEVICE)
            masks = masks_tr.to(DEVICE)
            optimizer.zero_grad()  # 초기화
            output = model(imgs)  # 예측
            loss = loss_fn(output, masks)  # 순전파
            loss.backward()  # 역전파
            optimizer.step()  # 학습
            iou = calculate_miou(masks.cpu().detach().numpy(), output.cpu())
            output = output.cpu().detach().numpy()
            masks = masks.cpu().detach().numpy()
            accuracy = pixel_accuracy(masks, output)  # mIou
            train_accs[idx].append(accuracy)
            train_ious[idx].append(iou)
            train_losses[idx] += loss.item()

    #### valid ####
    for model in models:
        model.eval()
    val_losses, val_accs, val_ious = [0] * len(models), [[] for _ in range(len(models))], [[] for _ in range(len(models))]
    with torch.no_grad():
        for images, masks_tr in tqdm(val_dataloader):
            for idx, model in enumerate(models):
                imgs = images[:,:,:,(idx)]
                imgs = np.reshape(imgs, (BATCH_SIZE,1,256,256)).to(DEVICE)
                masks = masks_tr.to(DEVICE)
                output = model(imgs)  # 예측
                print(output.shape, masks.shape)
                loss = loss_fn(output, masks)  # 순전파
                iou = calculate_miou(masks.cpu().detach().numpy(), output.cpu())
                output = output.cpu().detach().numpy()
                masks = masks.cpu().detach().numpy()
                accuracy = pixel_accuracy(masks, output)  # mIou
                val_accs[idx].append(accuracy)
                val_ious[idx].append(iou)
                val_losses[idx] += loss.item()

    for idx, (model, model_save_path, lr_scheduler) in enumerate(zip(models, model_save_paths, lr_schedulers)):
        train_acc = np.mean(train_accs[idx])
        train_iou = np.mean(train_ious[idx])
        val_acc = np.mean(val_accs[idx])
        val_iou = np.mean(val_ious[idx])
        val_loss = val_losses[idx] / len(val_dataloader)  # 평균 계산
        
        print(f"Model {idx+1} - EPOCH: {epoch}, TRAIN LOSS: {train_losses[idx]:.6f}, TRAIN mIou: {train_iou:.6f}, TRAIN ACC: {train_acc:.6f}, VAL LOSS: {val_loss:.6f}, VAL mIou: {val_iou:.6f}, VAL ACC: {val_acc:.6f}")
        
        lr_scheduler.step()
        
        if best_val_losses[idx] < val_iou:
            print(f"Model {idx+1} Save")
            best_val_losses[idx] = val_iou
            torch.save(model.state_dict(), model_save_path)
            early_stops[idx] = 0
        else:
            early_stops[idx] += 1
        
        # early stop
        if early_stops[idx] > 10:
            print(f"Early Stop for Model {idx+1}")
            break
        
print(f"Training finished for Model {idx+1}")

Train START
Epoch 1/100
--------------------


 35%|███▍      | 292/840 [09:51<17:09,  1.88s/it]

# Testing

In [ ]:
from transformers import SegformerImageProcessor
from datasets import load_dataset

hf_dataset_identifier = "segments/sidewalk-semantic"
ds = load_dataset(hf_dataset_identifier)
processor = SegformerImageProcessor()

id2label = {0: 'nofire', 1: 'fire'}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
MAX_PIXEL_VALUE = 65535 # 이미지 정규화를 위한 픽셀 최대값
def new_dataset(image):
    """ make new image (threshold)"""
    new_image = np.zeros_like(image[:, :, 0])
    for (prob,idx) in [(0.5, 6), (0.4, 5), (0.4, 1), (0.1, 0), (0.1, 4)]:
        new_image += prob * image[:, :, (idx)]
    return new_image
    
def get_img_762bands(path):
    image = tiff.imread(path)
    img = np.float32(image) / MAX_PIXEL_VALUE  # 정규화
    img = new_dataset(img)
    img = np.repeat(img[:, :, np.newaxis], 3, axis=2)
    #print(f"Image shape: {img.shape}") 
    return np.float32(img)

def preprocess_mask(mask):
    mask = torch.from_numpy(mask).float().unsqueeze(0)
    mask = F.interpolate(mask, size=(64, 64), mode='bilinear', align_corners=False)
    mask = mask.repeat(100, 1, 1, 1)  # 마스크를 100개로 복제
    mask = (mask > 0.5).float()  # 이진 마스크로 변환
    return mask

In [ ]:
from transformers import MobileViTForSemanticSegmentation
import tifffile as tiff
# MaskFormer 모델 구성 수정

model = MobileViTForSemanticSegmentation.from_pretrained("apple/deeplabv3-mobilevit-small")

num = 16266
tes = f'{BASE}/train_img/train_img_{num}.tif'
te = f'{BASE}/train_mask/train_mask_{num}.tif'
test_image = get_img_762bands(tes)
test_image = torch.from_numpy(test_image).float().unsqueeze(0)#.to(DEVICE)
test_image = test_image.permute(0, 3, 1, 2)
#print(test_image.shape)
output= model(test_image)

In [ ]:
import requests
import torch
from PIL import Image
from transformers import AutoImageProcessor, MobileViTModel, MobileViTConfig

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
config = MobileViTConfig(num_channels= 3, image_size = 256,)
image_processor = AutoImageProcessor.from_pretrained("apple/deeplabv3-mobilevit-small")
model = MobileViTModel(config= config)
model.return_dict = False
#model.pixel_values=[1, 3, 512, 512]
inputs = image_processor(images=image, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    print('output추론')

# logits are of shape (batch_size, num_labels, height, width)
logits = outputs

In [ ]:
outputs

In [ ]:
print(inputs['pixel_values'].shape)
plt.imshow(image)  # 흑백 이미지로 표시
plt.axis('off')  # 축 제거
plt.show()

In [ ]:
print(inputs['pixel_values'].shape)
plt.imshow(logits)  # 흑백 이미지로 표시
plt.axis('off')  # 축 제거
plt.show()

In [ ]:
logits.shape

In [ ]:
output.logits.ahep

In [ ]:
output.logits.ahep

# Inference

In [ ]:
y_pred_dict = {}

# data load
IMAGES_PATH = f'{BASE}/test_img/'
images_test = [os.path.join(IMAGES_PATH, image) for image in test_meta['test_img'] ]
masks_test = [os.path.join(MASKS_PATH, mask) for mask in test_meta['test_mask'] ]

test_dataset = CustomDataset(images_test, masks_test, mode = 'test')

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=1,
    shuffle=False
)


# load best model
# smp
#model = smp.FPN('timm-mobilenetv3_large_100', encoder_weights='imagenet', classes=1, activation=None,in_channels=1)
model.load_state_dict(torch.load(MODEL_SAVE))

model.to(DEVICE)
model.eval()
with torch.no_grad():
    for idx, imgs in enumerate(tqdm(test_dataloader)):
        name = test_meta['test_img'][idx]
        imgs = imgs.to(DEVICE)
        output = model(imgs)
        
        output = output.cpu()#.detach().numpy()
        output=F.sigmoid(output)
        #low = np.quantile(output, 0.9999)
        y_pred = np.where(output > 0.1, 1, 0) # 임계값 처리
        y_pred = np.reshape(y_pred,(256,256))
        
        y_pred = y_pred.astype(np.uint8)

        y_pred_dict[name] = y_pred
#
import joblib
joblib.dump(y_pred_dict, f'{BASE}/y_pred-7.pkl')